# Decision Trees

- Is a popular **non-parametric supervised ML algorithm that can be used for both classification and regression tasks**
- The tree itself is a model in decision trees and we would like to estimate an optimal tree structure from the given training data.
    - **The internal node contains conditions on features. Depending on the outcome of the comparision, we take an appropriate path in the tree. This process is continued till we reach a leaf node**
    - In case of classification, leaf nodes containing label and in case of regression, the prediction is obtained by taking sample mean of labels of the subset of training examples present in that leaf node.
    
We will implement decision tree for classification with **ID3 algorithm**


## Relevant Formulae 

### Impurity Measure


If we define $p_i,k$ to be the proportion of data points in node $i$ ($\mathcal{R}(i)$) assigned to class $k$, where $k = 1,...,K$, then :

$$
    p_{i,k} = \frac {1}{N_i} \sum \limits_{x^{(i)} \in \mathcal{R}_i} \mathcal {1}(y^{(i)} = k)
$$

where $N_i$ is the number of samples in region $\mathcal {R}_i$


### Misclassification Error

It is the proportion of misclassified examples in node $i$ :

$$
    Q_i(T) = 1 - p_{i,k(i)} = \frac {1}{N_i} \sum \limits_{x^{(i)} \in \mathcal{R}_i} \mathcal {1}(y^{(i)} \ne k)
$$

where $k(i) = arg_k max p_{i,k}$ = **prediction of class in node $i$**


### Gini Index

$$
    G_i = \sum \limits_{k=1}^{K} p_{i,k}(1 - p_{i,k})
$$



### Entropy

Entropy of node $i$ is given by :

$$
    H_i = - \sum \limits_{k=1}^{n} p_{i,k} (log_2 p_{i,k})
$$

**In ID3**, entropy is calculated for each remaining attribute. **The attribute with the smallest entropy is used to split the dataset on a given iteration.**


### Information Gain

- Information Gain (IG) is a **measure of the effectiveness** of an attribute in classifying the training data
- It is simply the **expected reduction in entropy** casusd by partitioning the examples according to the chosen attribute

Intuitively,

**IG(attribute) = Entropy of dataset - Entropy of attribute**

Formally, the information gain IG(S,A) of an attribute A, relative to collection of examples S, is defined as :

$$
    IG(S,A) = Entropy(S) - \sum \limits_{v \in Values(A)} \frac {|S_v|} {|S|} Entropy(S_v)
$$

<br>

where $Entropy(S)$ is the entropy of dataset, $\sum \limits_{v \in Values(A)} \frac {|S_v|} {|S|} Entropy(S_v)$ is the entropy of the attribute

In [4]:
import numpy as np
import pandas as pd

eps = np.finfo(float).eps
eps

2.220446049250313e-16

Here, `eps` is the smallest representable number. At times we get $log(0)$ or $0$ in the denominator. **To avoid that, we are going to use this.**

We will be using a synthetic dataset to demonstrate decision trees

In [5]:
data = [
    ['overcast', 'hot', 'high', 'FALSE', 'yes'],
    ['overcast', 'cool', 'normal', 'TRUE', 'yes'],
    ['overcast', 'mild', 'high', 'TRUE', 'yes'],
    ['overcast', 'hot', 'normal', 'FALSE', 'yes'],
    ['rainy', 'mild', 'high', 'FALSE', 'yes'],
    ['rainy', 'cool', 'normal', 'FALSE', 'yes'],
    ['rainy', 'cool', 'normal', 'TRUE', 'no'],
    ['rainy', 'mild', 'normal', 'FALSE', 'yes'],
    ['rainy', 'mild', 'high', 'TRUE', 'no'],
    ['sunny', 'hot', 'high', 'FALSE', 'no'],
    ['sunny', 'hot', 'high', 'TRUE', 'no'],
    ['sunny', 'mild', 'high', 'FALSE', 'no'],
    ['sunny', 'cool', 'normal', 'FALSE', 'yes'],
    ['sunny', 'mild', 'normal', 'TRUE', 'yes']
]

cols = ['outlook','temp','humidity','windy','play']

df = pd.DataFrame(data, columns=cols)

In [3]:
print(df.shape)
print(df.keys()[-1])

(14, 5)
play


In [4]:
df.values

array([['overcast', 'hot', 'high', 'FALSE', 'yes'],
       ['overcast', 'cool', 'normal', 'TRUE', 'yes'],
       ['overcast', 'mild', 'high', 'TRUE', 'yes'],
       ['overcast', 'hot', 'normal', 'FALSE', 'yes'],
       ['rainy', 'mild', 'high', 'FALSE', 'yes'],
       ['rainy', 'cool', 'normal', 'FALSE', 'yes'],
       ['rainy', 'cool', 'normal', 'TRUE', 'no'],
       ['rainy', 'mild', 'normal', 'FALSE', 'yes'],
       ['rainy', 'mild', 'high', 'TRUE', 'no'],
       ['sunny', 'hot', 'high', 'FALSE', 'no'],
       ['sunny', 'hot', 'high', 'TRUE', 'no'],
       ['sunny', 'mild', 'high', 'FALSE', 'no'],
       ['sunny', 'cool', 'normal', 'FALSE', 'yes'],
       ['sunny', 'mild', 'normal', 'TRUE', 'yes']], dtype=object)

In [13]:
print(df[df.keys()[-1]])

0     yes
1     yes
2     yes
3     yes
4     yes
5     yes
6      no
7     yes
8      no
9      no
10     no
11     no
12    yes
13    yes
Name: play, dtype: object


In [6]:
def find_entropy_whole(df) :
    
    # Last column is the target variable
    target = df.keys()[-1]
    
    # Initialization
    overall_entropy = 0
    
    # Possible values of target
    values_in_target = df[target].unique()
    
    for val in values_in_target :
        print(val)
        p = df[target].value_counts()[val] / len(df[target])
        print(p)
        overall_entropy += (-p * np.log2(p))
    
    return overall_entropy

In [7]:
find_entropy_whole(df)

yes
0.6428571428571429
no
0.35714285714285715


0.9402859586706311

In [5]:
def find_entropy_whole(df) :
    
    # Last column is the target variable
    target = df.keys()[-1]
    
    # Initialization
    overall_entropy = 0
    
    # Possible values of target
    values_in_target = df[target].unique()
    
    for val in values_in_target :
        p = df[target].value_counts()[val] / len(df[target])
        overall_entropy += (-p * np.log2(p))
    
    return overall_entropy



def find_entropy_of_attribute(df, attribute) :
    # Last column is the target variable
    target = df.keys()[-1]
    
    # Initialization
    entropy_attribute = 0

    
    # Possible values of target
    values_in_target = df[target].unique()
    
    # Gives different features in the passed in attribute. For eg. 'hot', 'cold', in temperature, etc.
    values_in_attr = df[attribute].unique()
    
        
    for val_attr in values_in_attr :
        overall_entropy = 0

        for val_target in values_in_target :
            num = len(df[attribute][df[attribute] == val_attr][df[target] == val_target])
            den = len(df[attribute][df[attribute] == val_attr])
            p = num / (den + eps)
            overall_entropy += -p * np.log2(p + eps)
        p2 = den / len(df)
        entropy_attribute += -p2 * overall_entropy
    return abs(entropy_attribute)

def find_best_attribute_to_divide(df):
    IG = []

    all_attributes = df.keys()[:-1]
    
    for attribute in all_attributes:
        IG.append(find_entropy_whole(df) - find_entropy_of_attribute(df, attribute))
    index_of_attribute_with_max_IG = np.argmax(IG)
    best_attribute = all_attributes[index_of_attribute_with_max_IG]
    
    return best_attribute

In [6]:
for attribute in df.keys()[:-1]:
    print(f'Entropy of the attribute "{attribute}" is :', find_entropy_of_attribute(df, attribute))

Entropy of the attribute "outlook" is : 0.6935361388961914
Entropy of the attribute "temp" is : 0.9110633930116756
Entropy of the attribute "humidity" is : 0.7884504573082889
Entropy of the attribute "windy" is : 0.892158928262361


In [7]:
find_best_attribute_to_divide(df)

'outlook'

In [8]:
def buildTree(df, tree=None) :
    target = df.keys()[-1]
    node = find_best_attribute_to_divide(df)
    attValue = np.unique(df[node])
    print(node)
    if tree is None :
        tree = {}
        tree[node] = {}

    for value in attValue :
        subtree = df[df[node] == value].reset_index(drop=True)
        clValue, counts = np.unique(subtree['play'], return_counts=True)

        # Checking purity of subset
        if len(counts) == 1: 
            # Leaf Node
            tree[node][value] = clValue[0]
        else:
            # If not a leaf node, call the function recursively
            tree[node][value] = buildTree(subtree) 
    
    return tree

In [9]:
buildTree(df)

outlook
windy
humidity


{'outlook': {'overcast': 'yes',
  'rainy': {'windy': {'FALSE': 'yes', 'TRUE': 'no'}},
  'sunny': {'humidity': {'high': 'no', 'normal': 'yes'}}}}